In [1]:
import os, sys
MYPROJECT = '/Users/ryuki/Projects/graduation_research'
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")

import django
django.setup()

In [3]:
from review_based_recommender.models import Spot, Review, City
import django_pandas

In [5]:
from django_pandas.io import read_frame
reviews = Review.objects.all()
review_df = read_frame(reviews)

In [6]:
review_df

,id,username,title,content,rating,spot
0,1,niramechan,ボードウォーク閉鎖中,観音崎京急ホテルやその施設を利用するなら京急の馬堀海岸から送迎バスが出ています。ボードウォー...,30,"spot=観音崎 (横須賀市), count=73/73"
1,2,yukof186,ハイキングでした…,美術館から足をのばして行きましたが、思ったより急な登りが沢山。軽い散歩のつもりでいったら、わ...,40,"spot=観音崎 (横須賀市), count=73/73"
2,3,隆幸 末,楽しかった！,車は観音崎第一駐車場に止めました。海のすぐそばです。平日は無料のようです。駐車場から遊歩道を...,50,"spot=観音崎 (横須賀市), count=73/73"
3,4,さすらいのとんしゅけ,海好きにはたまりません,ここのオススメは浦賀水道を通るお船を眺める事。一日にものすごい数の船が通るのです。タンカーや...,50,"spot=観音崎 (横須賀市), count=73/73"
4,5,ya1201,海岸散歩,京急ホテル宿泊の時はいつも朝海岸散歩します。\n東京湾入口行き交う船を間近に見ながらの散歩は...,40,"spot=観音崎 (横須賀市), count=73/73"
5,6,bjnoche,爽やかな潮風、自然での戯れ、モダンな施設、おいしい食事、、、癒しの場,観音崎は都心から最も近い自然浜のあるビーチリゾートとして人気があり、また地元の人にも海水浴や...,50,"spot=観音崎 (横須賀市), count=73/73"
6,7,とよとよ25,なかなか楽しめます,横須賀の南にある岬で、中に入れる灯台や戦跡が残されており、観光地としてお勧めです。このあたり...,40,"spot=観音崎 (横須賀市), count=73/73"
7,8,yoitoko137,東京湾に突き出した岬,三浦半島にある東京湾に突き出した岬で、東京湾を行き来する船舶がよく見える場所です。歴史のある...,40,"spot=観音崎 (横須賀市), count=73/73"
8,9,Adam756,夏場はBBQ客がいっぱい,横須賀中央駅から京急バスで30分ぐらいで着きました。海水浴、BBQ、灯台を見に行ったり自然遊...,40,"spot=観音崎 (横須賀市), count=73/73"
9,10,ずんだれ,横須賀を代表する景勝地,横須賀市歌は「白波は白波は岬にくだけ、光る風光る風大地に遊ぶ」という歌詞で始まりますが、まさ...,40,"spot=観音崎 (横須賀市), count=73/73"


In [22]:
review_df.groupby('spot').mean().sort_values('rating', ascending=False)

,id,rating
spot,,
"spot=ハヤトハコネ サロンアンドスパ (箱根町), count=1/1",71756.0,50.000000
"spot=YOKO CHAN本店 (港区), count=2/2",154791.0,50.000000
"spot=安平神社 (中央区), count=1/1",202677.0,50.000000
"spot=ホーンテッド トウキョウツアーズ (台東区), count=2/2",210742.5,50.000000
"spot=サクラテラス (墨田区), count=1/1",194371.0,50.000000
"spot=ホーストレッキングファーム三浦海 (三浦市), count=6/4",89734.0,50.000000
"spot=東京新宿射的場 (新宿区), count=2/2",186971.5,50.000000
"spot=リサイクル横丁 (横浜市), count=1/1",28816.0,50.000000
"spot=サイクリングホリデー東京 (千代田区), count=12/12",163611.5,50.000000


<bound method NDFrame.first of             id        username                               title  \
0            1      niramechan                          ボードウォーク閉鎖中   
1            2        yukof186                           ハイキングでした…   
2            3            隆幸 末                              楽しかった！   
3            4      さすらいのとんしゅけ                         海好きにはたまりません   
4            5          ya1201                                海岸散歩   
5            6         bjnoche  爽やかな潮風、自然での戯れ、モダンな施設、おいしい食事、、、癒しの場   
6            7          とよとよ25                           なかなか楽しめます   
7            8      yoitoko137                          東京湾に突き出した岬   
8            9         Adam756                        夏場はBBQ客がいっぱい   
9           10            ずんだれ                         横須賀を代表する景勝地   
10          11       minato818                 低地ですが、真昼は景色も壮観でさわやか   
11          12           ゆうき な                    アジサイと豪華客船が見られる散歩   
12          13        kumi8800                   砂浜で貝殻や石を拾っ